In [1]:
# Loading up necessary files
import myfitnesspal as mfp
import keyring
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date, timedelta, datetime
import ast
import re


KEYRING_SYSTEM = 'python-myfitnesspal://myfitnesspal-password'
mfp.keyring_utils.store_password_in_keyring("mccombkp", "2Interactive+")
client = mfp.Client('mccombkp')

In [2]:
# selects the desired date
string = str(client.get_date(2019, 3, 2))
date = string[0:8]
date = [date]

#grabs the macro nutrient breakdown for that desired date
nutrition = string[10:-1]
line2 = nutrition
temp1 = re.findall(r'\d+', line2) # through regular expression
nutrition = list(map(int, temp1))

complete_list = date + nutrition
single = pd.DataFrame([complete_list], columns=['Date', 'Calories', 'Carbohydrates', 'Fat', 'Protein', 'Sodium', 'Sugar'])
single

,Date,Calories,Carbohydrates,Fat,Protein,Sodium,Sugar
0,03/02/19,2632,136,141,191,1325,15


In [3]:
t = datetime.today()


#figure out how to simplefy this
thistuple = (t,
             t - timedelta(days=1),
             t - timedelta(days=2), 
             t - timedelta(days=3),
             t - timedelta(days=4),
             t - timedelta(days=5),
             t - timedelta(days=6))


output = []
# output2 = []
for x in thistuple:
    string = str(client.get_date(x))
    date = string[0:8]
    date = [date]
#     output1.append(date)
#grabs the macro nutrient breakdown for that desired date
    nutrition = string[10:-1]
    line2 = nutrition
    temp1 = re.findall(r'\d+', line2) # through regular expression
    nutrition = list(map(int, temp1))
    complete_list = date + nutrition
    output.append(complete_list)
    
    
    
multi = pd.DataFrame(output, columns=['Date', 'Calories', 'Carbohydrates', 'Fat', 'Protein', 'Sodium', 'Sugar'])
multi


,Date,Calories,Carbohydrates,Fat,Protein,Sodium,Sugar
0,10/22/19,2432,177,80,250,1683,66
1,10/21/19,2514,190,96,230,1592,32
2,10/20/19,2517,183,93,252,1097,77
3,10/19/19,1750,66,80,189,1413,14
4,10/18/19,2431,189,75,238,3122,35
5,10/17/19,2569,273,71,234,1908,34
6,10/16/19,2787,212,97,271,1448,112


In [4]:
#benchmark
#Ok thats great for 7 day rolling... how about for the entire week?


t = datetime.today()


#figure out how to simplefy this
thistuple2 = (t,
             t - timedelta(days=1),
             t - timedelta(days=2), 
             t - timedelta(days=3),
             t - timedelta(days=4),
             t - timedelta(days=5),
             t - timedelta(days=6))


output = []
# output2 = []
for x in thistuple2:
    string = str(client.get_date(x))
    date = string[0:8]
    date = [date]
#     output1.append(date)
#grabs the macro nutrient breakdown for that desired date
    nutrition = string[10:-1]
    line2 = nutrition
    temp1 = re.findall(r'\d+', line2) # through regular expression
    nutrition = list(map(int, temp1))
    complete_list = date + nutrition
    output.append(complete_list)
        
multi2 = pd.DataFrame(output, columns=['Date', 'Calories', 'Carbohydrates', 'Fat', 'Protein', 'Sodium', 'Sugar'])

#
multi2['Date'] = pd.to_datetime(multi2['Date'])    
multi2['Week'] = multi2.Date.apply(lambda x: x.strftime('%W'))
dftemp = multi2['Week'].iloc[0]
date_this_week = multi2[multi2['Week'] == dftemp]    

date_this_week

,Date,Calories,Carbohydrates,Fat,Protein,Sodium,Sugar,Week
0,2019-10-22,2432,177,80,250,1683,66,42
1,2019-10-21,2514,190,96,230,1592,32,42


In [5]:
#Data Frames To work With:
#Single: A loo at one specific day's macros (largely irrelevant)
#multi: A 7 day rolling look at nutrition (unique)
# Date_this_week: a look at the weeks nutrtion that has already happened and what ahead (good for planning)'

In [6]:
#let's set some guidelines
#BMR is the estimated calories burned in a day by default... meaning if you just laid in bed all day and didnt move, this is the estimated calories you were gaurenteed to burn
BMR = 2189
#TDEE is the estimated calories that you burn that include your BMR, but also other variables like:
#1.) Exercise for the day
#2.) Thermic Effect of Food: explain this..
#3.) N.E.A.T: non exercise related movement, like steps taken a day, or random twitching 
TDEE = 3000

#Macro Nutrition Split: Macros are what make up calories. They are Protein (4 calories per 1 gram), Carbohydrates (4 calories per gram), and fat (9 calories per 1 gram).
#You should set a <breakdown here>
print('Weekly Caloric Goal For Maintenance:')
print(TDEE * 7)

Weekly Caloric Goal For Maintenance:
21000


In [7]:
print('Caloric Level For Past 7 Days:')
if TDEE * 7 > multi.Calories.sum():
    print('You Are ' + str(TDEE * 7 - multi.Calories.sum()) + ' Under Maintenance')
elif TDEE * 7 < multi.Calories.sum():
    print('You Are ' + str(TDEE * 7 - multi.Calories.sum()) + ' Over Maintenance')
else:
    print('You are at Maintenance')

print ('')
print("You're Caloric Intake Relative to Maintenance:")
print(round(multi.Calories.sum() / (TDEE * 7),2))
print('')
print('Percent of Calories From Protein')
print(round((multi.Protein.sum() * 4) / (multi.Calories.sum()),2))
print('')
print('Percent of Calories From Fat')
print(round((multi.Fat.sum() * 9) / (multi.Calories.sum()),2))
print('')
print('Percent of Calories From Carbohydrates')
print(round((multi.Carbohydrates.sum() * 4) / (multi.Calories.sum()),2))

Caloric Level For Past 7 Days:
You Are 4000 Under Maintenance

You're Caloric Intake Relative to Maintenance:
0.81

Percent of Calories From Protein
0.39

Percent of Calories From Fat
0.31

Percent of Calories From Carbohydrates
0.3


In [120]:
# 7 day Rolling graphs

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
# import plotly.plotly as py
import plotly.express as px



# animals=['giraffes', 'orangutans', 'monkeys']

# fig = go.Figure([go.Bar(x=animals, y=[20, 14, 23])])

TDEE_7 = 3000 * 7
Total_Calories_7 = multi.Calories.sum()
calories_remaining = TDEE_7 - multi.Calories.sum()
caloric_surplus = multi.Calories.sum() - TDEE_7


x = ['TDEE_7','Total_Calories_7']
# colors[1] = 'none'


print("You're 7 Day Rolling Nutritional Breakdown:")
print('')

print('With an Estimated 7 Day TDEE at ' + str(TDEE_7) + ', You are Around ' + str((round(multi.Calories.sum() / (TDEE * 7),2))) + ' Percent of a Caloric Intake Required to Maintain Weight.')


if TDEE_7 > multi.Calories.sum():

    fig = go.Figure(data=[
        go.Bar(name='Calories Consumed This Week', x=x, y=[Total_Calories_7],
              hovertext=['Calories Consumed']),
        go.Bar(name='Calories Remaining', x=x, y=[calories_remaining],
              marker_color='whitesmoke',
              hovertext=['Calories Under Weekly TDEE'])
    ])
    # Change the bar mode
    fig.update_layout(barmode='stack')


    fig.update_layout(
        autosize=False,
        width=600,
        height=600,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),)

    fig.update_layout(
        title = 'Caloric Goals This Week'
        )

    fig.show()


# ^ Good!  
        
elif TDEE_7 < multi.Calories.sum():
        
    fig = go.Figure(data=[
        go.Bar(name='Calories Consumed This Week', x=x, y=[Total_Calories_7],
              marker_color='blue',
              hovertext=['Calories Within Limit']),
        go.Bar(name='Calories Over Maintenance', x=x, y=[caloric_surplus],
              marker_color='red',
              hovertext=['Calories Over Limit'])
    ])
    # Change the bar mode
    fig.update_layout(barmode='stack')


    fig.update_layout(
        autosize=False,
        width=600,
        height=600,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),)

    fig.update_layout(
        title = 'Caloric Goals This Week'
        )

    fig.show()

else: print ('You Are At a perfect maintenance')
    
    
    

    
    
    
    
    
    
    
    
blah = date_this_week.groupby(['Week']).agg({'Protein': pd.Series.sum,
                             'Fat': pd.Series.sum,
                             'Carbohydrates': pd.Series.sum}).reset_index()

blah['Protein'] = blah['Protein'] * 4
blah['Fat'] = blah['Fat'] * 9
blah['Carbohydrates'] = blah['Carbohydrates'] * 4
blah    
    
    
    
listt = [multi.Protein.sum() * 4, multi.Carbohydrates.sum() * 4, multi.Fat.sum() * 9]

colors = ['Green', 'Blue', 'darkorange']

fig2 = go.Figure(data=[go.Pie(labels=['Protein','Carbohydrates','Fat'], 
                             values=listt)])
fig2.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))


fig2.update_layout(
    autosize=False,
    width=600,
    height=600,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),)


fig2.update_layout(
    title_text="Macronutrition Breakdown for the Past 7 Days")
#     # Add annotations in the center of the donut pies.
#     annotations=[dict(text='GHG', x=0.18, y=0.5, font_size=20, showarrow=False),
#                  dict(text='CO2', x=0.82, y=0.5, font_size=20, showarrow=False)])

fig2.show()

You're 7 Day Rolling Nutritional Breakdown:

With an Estimated 7 Day TDEE at 21000, You are Around 0.81 Percent of a Caloric Intake Required to Maintain Weight.


In [126]:
# This Weeks look


TDEE_7 = 3000 * 7
Total_Calories_7 = date_this_week.Calories.sum()
calories_remaining = TDEE_7 - date_this_week.Calories.sum()
caloric_surplus = date_this_week.Calories.sum() - TDEE_7


x = ['TDEE_7','Total_Calories_7']
# colors[1] = 'none'


print("You're 7 Day Rolling Nutritional Breakdown:")
print('')

print('With an Estimated 7 Day TDEE at ' + str(TDEE_7) + ', You are Around ' + str((round(date_this_week.Calories.sum() / (TDEE * 7),2))) + ' Percent of a Caloric Intake Required to Maintain Weight So Far This Week.')
print('')



fig = go.Figure(data=[
    go.Bar(name='Calories Consumed This Week', x=x, y=[Total_Calories_7],
             hovertext=['Calories Consumed']),
    go.Bar(name='Calories Remaining', x=x, y=[calories_remaining],
            marker_color='whitesmoke',
            hovertext=['Calories Under Weekly TDEE'])
    ])
    # Change the bar mode
fig.update_layout(barmode='stack')


fig.update_layout(
        autosize=False,
        width=600,
        height=600,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),)

fig.update_layout(
        title = 'Caloric Goals This Week'
        )

fig.show()




listt = [date_this_week.Protein.sum() * 4, date_this_week.Carbohydrates.sum() * 4, date_this_week.Fat.sum() * 9]

colors = ['Green', 'Blue', 'darkorange']

fig2 = go.Figure(data=[go.Pie(labels=['Protein','Carbohydrates','Fat'], 
                             values=listt)])
fig2.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))


fig2.update_layout(
    autosize=False,
    width=600,
    height=600,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),)


fig2.update_layout(
    title_text="Macronutrition Breakdown for this Week Thus Far")
#     # Add annotations in the center of the donut pies.
#     annotations=[dict(text='GHG', x=0.18, y=0.5, font_size=20, showarrow=False),
#                  dict(text='CO2', x=0.82, y=0.5, font_size=20, showarrow=False)])

fig2.show()




You're 7 Day Rolling Nutritional Breakdown:

With an Estimated 7 Day TDEE at 21000, You are Around 0.24 Percent of a Caloric Intake Required to Maintain Weight So Far This Week.

